# NFT image and traits generator

This script used to build images from `image-sources` folder with predefined metadata and traits per NFT. You can use `./metadata/all-traits.json` (output from `generator.ipynb`) as input for this script. Useful when you need to handle and regenerate large amounts of images and dont need random traits.

In [11]:
from PIL import Image 
from IPython.display import display 
import random
import json

In [12]:
# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%
AllTraits = [
    {
        "name": "Hair",
        "traits":  ["hair1", "hair2", "hair3", "hair4"],
        "weights": [30, 20, 10, 40],
        "folder": "hair"
    },
    {
        "name": "Accessory",
        "traits":  ["accessory1", "accessory2", "accessory3", "accessory4"] ,
        "weights": [25, 25, 25, 25],
        "folder": "accessory"
    },
    {
        "name": "Face",
        "traits":  ["face1", "face2", "face3", "face4"],
        "weights": [25, 25, 25, 25],
        "folder": "face"
    },
    {
        "name": "Body",
        "traits":  ["Duck"],
        "weights": [100],
        "files": ["Duck"],
        "folder": "body"
    },
    {
        "name": "Background",
        "traits":  ["Green","Yellow","Blue","White"],
        "weights": [30, 10, 20, 40],
        "folder": "background"
    }
]

In [13]:
## Load traits

TOTAL_IMAGES = 32 # Number of random unique images we want to generate

all_images = [] 

# do check weights
for trait in AllTraits:
    print(trait["name"] +": t: "+ str(len(trait["traits"]))+ ", w: " + str(len(trait["weights"])))

#### Generate Metadata for all Traits 
METADATA_FILE_NAME = './metadata/all-traits.json'; 
    
with open(METADATA_FILE_NAME) as json_file:
    all_images = json.load(json_file)


counts = {}

for trait in AllTraits:
    counts[trait["name"]] = {}
    for item in trait["traits"]:
        counts[trait["name"]][item] = 0

for image in all_images:
    for k,c in counts.items():
        counts[k][image[k]] += 1

print("=====")

for k in counts:
    print(k + ": ")
    print(counts[k])

Hair: t: 4, w: 4
Accessory: t: 4, w: 4
Face: t: 4, w: 4
Body: t: 1, w: 1
Background: t: 4, w: 4
=====
Hair: 
{'hair1': 5, 'hair2': 8, 'hair3': 1, 'hair4': 18}
Accessory: 
{'accessory1': 5, 'accessory2': 12, 'accessory3': 7, 'accessory4': 8}
Face: 
{'face1': 7, 'face2': 10, 'face3': 6, 'face4': 9}
Body: 
{'Duck': 32}
Background: 
{'Green': 11, 'Yellow': 2, 'Blue': 6, 'White': 13}


In [14]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [15]:
# load an array of all images
trait_img = {}
for trait in AllTraits:
    trait_img[trait["name"]] = {}
    for item in trait["traits"]:
        trait_img[trait["name"]][item] = Image.open(f'./image-source/{trait["folder"]}/{item}.png').convert('RGBA')
        
#watermark = Image.open(f'./watermark.png').convert('RGBA')

#### Generate Images    
for item in all_images:

    im = Image.new('RGBA', (32,32))
    
    for trait in reversed(AllTraits):
        this_layer = trait_img[trait["name"]][item [ trait["name"] ]]
        im = Image.alpha_composite(im, this_layer)


    # WATERMARK
    
    #im = Image.alpha_composite(im, watermark)

    #Resize
    im = im.resize((128,128), Image.Resampling.NEAREST)

    #Convert to RGB
    im = im.convert('RGB')
    file_name = str(item["tokenId"]) + ".jpg"
    im.save("./images-output/" + file_name, quality=95)
    